# RUZ_search_for_blanks_in_timetable

*Авторы:
Кармадонова О., Межакова П.*


Перед Вами программа, которая автоматически сопоставляет расписание нескольких людей из базы РУЗ ВШЭ и указывает промежутки, в которые все перечисленные люди свободны от учебных занятий.
Она поможет вам и вашим коллегам из Вышки найти свободное окошко в рабочем графике по любому поводу - будь то мероприятие студенческой организации или встреча с научным руководителем :)

Ссылка на сайт РУЗа: https://ruz.hse.ru/

Для работы программы Вам **необходимы** следующие пакеты для Python: *Selenium*, *Beautiful Soup* и *Pandas*.
Помимо этого, будет использоваться модуль time.

Помимо установки пакета Selenium, Вам также понадобится предустановленный *Google Chrome версии .95 и выше*, а также скачанный на компьютер *WebDriver и сохранённый к нему путь*: скачать WebDriver можно по [ссылке](https://chromedriver.chromium.org/downloads), путь к файлу можно посмотреть и скопировать в Вашем файловом менеджере.

### Шаг -1. Установка пакетов

Если вдруг какие-либо из вышеперечисленных пакетов у Вас не установлены, Вы можете установить их, запустив соответствующие ячейки ниже.
Если они у Вас установлены или предустановлены, пропустите данные ячейки.

In [ ]:
!pip install selenium
!pip install BeautifulSoup
!pip install pandas

### Шаг 0. Импорт пакетов и запуск Selenium WebDriver

Если Вы удостоверились, что все необходимые пакеты и программы у Вас установлены/скачаны, то для начала работы запустите следующую ячейку.

Программа запросит у Вас путь к Selenium WebDriver, укажите его в формате "C:\Users\user\Desktop\chromedriver.exe"

*Если на данном этапе возникла ошибка и Selenium WebDriver (окошко Google Chrome с надписью "Браузером Chrome управляет автоматизированное тестовое ПО") не запустился, перепроверьте, пожалуйста, корректно ли указан путь.*

**Важно для пользователей MacOS!** Обратите внимание на направление слэшей (чёрточек), если они у вас указаны как "/", то, пожалуйста, замените их на "\\".

In [1]:
from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
time_dict = {"09:30" : 64, "11:10" : 32, "13:00" : 16, "14:40" : 8, "16:20" : 4, "18:10" : 2, "19:40" : 1}

user_br = input("Пожалуйста, введите или скопируйте в данную ячейку путь к Selenium WebDriver: ")

br = wb.Chrome(user_br)

Пожалуйста, введите или скопируйте в данную ячейку путь к Selenium WebDriver: C:\\Users\\Полина\\YandexDisk\\Учеба\\3 курс\\Питон\\chromedriver_win32\\chromedriver.exe


<ipython-input-1-164535f510c6>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  br = wb.Chrome(user_br)


### Шаг 1. Ввод дат

В следующей ячейке кода программа запросит у Вас даты, которые Вы бы хотели проверить.

* Введите день, в течение которого Вы бы хотели провести встречу, в формате дд.мм.гггг (например: 06.12.2021) и нажмите Enter.
* После этого Вы также можете последовательно ввести другие интересующие Вас даты, программа выдаст свободное от пар время в каждый из них.
* Когда Вы укажете все желаемые даты - введите слово "конец" (без кавычек) и нажмите Enter, чтобы завершить ввод.

*Программа считывает каждый день отдельно, т. е., например, если Вам необходимо найти свободное время на неделе с 20.12.2021 по 26.12.2021, Вам нужно отдельно ввести дату каждого дня недели: 20.12.2021, 21.12.2021, 22.12.2021, 23.12.2021, 24.12.2021, 25.12.2021 и 26.12.2021, нажимая Enter после ввода каждой даты.*

*Даты могут идти не подряд: например, возможно запросить расписание только на 27.12.2021, 10.01.2022 и 17.02.2022.*

*Пожалуйста, помните о том, что в Вышке расписание формируется в начале каждого модуля, т. е. расписание на отдалённую дату может быть некорректным. Во избежание ошибок рекомендуем вводить даты не далее текущего модуля.*

*В случае ввода даты в неправильном формате программа будет просить Вас ввести дату еще раз.*

Если Вы готовы, запустите, пожалуйста, следующую ячейку.

In [3]:
dati = []

while dati == []:
    udate = str(input("Ваша дата: "))
    while udate != "конец": 
        if len(udate) != 10:
            print("Ошибка! Неправильный формат даты")
        elif udate[2] != "." and udate[5] != ".":
            print("Ошибка! Неправильный формат даты")
        udatelist = udate.split(".")
        udatelist.append(1)
        udatelist.append(1)
        
        #проверка даты на лимиты значений, нет проверки, что пользователи ввели несуществующую дату (например, 30.02)
        day = int(udatelist[0])
        month = int(udatelist[1])
        year = int(udatelist[2])
        if 1 > day:
            print("Ошибка! Вы ввели несуществующий день")
        elif day > 31:
            print("Ошибка! Вы ввели несуществующий день")
        elif 1 > month:
            print("Ошибка! Вы ввели несуществующий месяц")
        elif month > 12:
            print("Ошибка! Вы ввели несуществующий месяц")
        else:
            dati.append(udate)
        udate = str(input("Пожалуйста, введите следующую дату или 'конец', если готовы завершить ввод.\nВаша дата: "))

time.sleep(3)

Ваша дата: 24.01.2022
Пожалуйста, введите следующую дату или 'конец', если готовы завершить ввод.
Ваша дата: 25.01.2022
Пожалуйста, введите следующую дату или 'конец', если готовы завершить ввод.
Ваша дата: конец


### Шаг 2. Ввод имён

В следующей ячейке Вам нужно будет указать людей, для которых Вам нужно проверить расписание.

* Сперва введите ФИО первого человека, для которого хотите проверить расписание и нажмите Enter.

* В следующей ячейке укажите значение для должности данного человека в Вышке: 1, если это студент_ка и 2, если это преподаватель_ница и нажмите Enter.

* После этого повторите ввод для всех остальных людей, для которых хотите запросить информацию.

* Когда список людей закончится, завершите ввод, указав слово "конец" и нажав Enter.

**Важно! Если на 2 шаге (вводе значения для должности) Вы ввели значение, отличное от 1 и 2, программа выдаст ошибку на последующих шагах.
Будьте внимательны.**

**Если ошибка допущена - прервите выполнение данной ячейки и запустите её снова.**

*Допустимо вводить только фамилию и имя, но это сработает корректно только в случае, если в Вышке больше нет людей с такими же фамилией и именем. Если Вы не уверены в этом - рекомендуем вводить полное ФИО.*

In [4]:
user_f = ""
user_type_f = ""

while user_f == "":
    user = input("Введите ФИО коллеги:")
    user_f = ""
    user_type_f = ""
    while user != "конец":
        user_type = str(input("Введите 1 для студент_ки и 2 для преподаватель_ницы: "))
        user_f = user_f + user + ";"
        user_type_f = user_type_f + user_type + ";"
        user = str(input("Введите ФИО следующе_й коллеги либо 'конец' для завершения ввода:"))
    
#РУЗ читает фамилии, даже если они все с маленькой буквы, все капсом или ДаЖе в ТаКоМ ВиДе, 
#поэтому приводить их к единому стилю не обязательно 
user_f = user_f.strip(";").split(";")
user_type_f = user_type_f.strip(";").split(";")
inner_dict = {user_f[i]:user_type_f[i] for i in range (len(user_f))}
grand = {} #словарь, где ключами будут даты, а значениями словари, в которых ключами будут - ФИО, а значениями - десятичные числа: они содержат в себе информацию о расписании человека (см. словарь "time_dict", там для каждой пары присвоено значение")
grand = grand.fromkeys(dati)
users = {} 
users = users.fromkeys(user_f)
for key in grand: 
    grand[key] = users.copy()

Введите ФИО коллеги:Межакова Полина
Введите 1 для студент_ки и 2 для преподаватель_ницы: 1
Введите ФИО следующе_й коллеги либо 'конец' для завершения ввода:Баранов Андрей Леонидович
Введите 1 для студент_ки и 2 для преподаватель_ницы: 1
Введите ФИО следующе_й коллеги либо 'конец' для завершения ввода:Морозов Данила
Введите 1 для студент_ки и 2 для преподаватель_ницы: 1
Введите ФИО следующе_й коллеги либо 'конец' для завершения ввода:Иншаков Илья Александрович
Введите 1 для студент_ки и 2 для преподаватель_ницы: 2
Введите ФИО следующе_й коллеги либо 'конец' для завершения ввода:конец


### Шаг 3. Магия!

Замечательно, всё готово к работе!

Теперь Вам остаётся только запустить следующую ячейку и подождать некоторое время, пока программа подготовит результаты работы :)

**Примечания:**

* *Из-за особенностей работы Selenium WebDriver под ячейкой появится несколько предупреждений на красном фоне, напоминающих ошибки, но на работоспособность программы это не влияет.*

* *Время ожидания может составлять от нескольких секунд до нескольких минут. Чем больше дат и людей было введено - тем, к сожалению, дольше придётся подождать.*

In [6]:
url = 'https://ruz.hse.ru/ruz/schedule'
br.get(url)
time.sleep(5)

for i in dati: 
    #вставка дат в рамках парсера удается по какой-то причине лишь со 2-го раза, but let it work this way
    #вставляем дату в графу "с" на сайте
    br.find_element_by_css_selector('#start').click() 
    br.find_element_by_css_selector('#start').clear() 
    
    br.find_element_by_css_selector('#start').send_keys(i)
    br.find_element_by_css_selector('#start').clear() 
    br.find_element_by_css_selector('#start').send_keys(i) #наконец успешно вставляем дату
    time.sleep(2)
    
    #вставляем дату "по" на сайте
    br.find_element_by_css_selector('#end').click()
    br.find_element_by_css_selector('#end').clear()
    
    br.find_element_by_css_selector('#end').send_keys(i) 
    br.find_element_by_css_selector('#end').clear()
    br.find_element_by_css_selector('#end').send_keys(i) #наконец успешно вставляем дату
    time.sleep(2)
    
    for j in range(len(user_f)): 
        #нажмем на тип человека на сайте
        if user_type_f[j] == "1":
            br.find_element_by_css_selector('body > ruz-root > div > ruz-schedule-index > ruz-table > div > div.btn-group.mt-3.mr-3.d-none.d-sm-block > button:nth-child(2)').click()
            #теперь ФИО 
            #сначала очистим поле после предыдущего человека 
            br.find_element_by_css_selector('#autocomplete-student').clear()
            #вставим ФИО в поле 
            br.find_element_by_css_selector('#autocomplete-student').send_keys(user_f[j])
            #последний раз нажать 
            time.sleep(2)
            br.find_element_by_css_selector('#pr_id_1_list > li').click()
            #все работает, теперь парсим таблицу с расписанием для студента   
        else: 
            br.find_element_by_css_selector('body > ruz-root > div > ruz-schedule-index > ruz-table > div > div.btn-group.mt-3.mr-3.d-none.d-sm-block > button:nth-child(3)').click()
            #теперь ФИО 
            #сначала очистим поле после предыдущего человека 
            br.find_element_by_css_selector('#autocomplete-lecturer').clear()
            #вставим ФИО в поле 
            br.find_element_by_css_selector('#autocomplete-lecturer').send_keys(user_f[j])
            #последний раз нажать
            time.sleep(2)
            br.find_element_by_css_selector('#pr_id_1_list > li').click()
        time.sleep(2)
        
        soup = bs(br.page_source)
        time_b = soup.find_all('td', attrs = {'class': 'beginLesson ng-star-inserted'}) 
        couples = 0 
        for s in range(len(time_b)): 
            time_e = time_b[s].find('div').get_text()
            couples = couples + time_dict[time_e] 
            #couples - это сумма всех десятичных значений для каждого человека (см. словарь "time_dict")
        grand[i][user_f[j]] = couples #значения couples записали в словарь grand

extra = {} 
#словарь, в котором ключи - даты, а значения - результат побитового сравнения сумм десятичных значений, 
#переведенных в двоичную систему систему счисления

extra = extra.fromkeys(dati)
for i in dati: 
    temp = 0
    for j in range(len(user_f)): 
        temp = grand[i][user_f[j]] | temp #побитовое сравнение
    extra[i] = temp

for k, v in extra.items(): #перевод из десятичной в двоичную в строку с длиной 7 (т.к. 7 пар физиически существуют в расписании)
    b = ''
    a = v
    for i in range(7): 
        b = str(a % 2) + b
        a = a // 2
    extra[k] = b
#в значениях extra: 0 - эта пара свободна, 1 - эта пара занята 

#здесь мы ходим по ключам и значениям словаря extra, каждый индекс значения проверяется на 0/1
#учтен вывод случая, когда все рассматриваемые студенты_тки/преподаватели)ницы заняты 

dates_df = []

for k, v in extra.items():
    z = 0
    final = ""
    for i in range(len(v)): 
        if v[i] == "0":
            z = z + 1
            final = final + str(i+1) + "-ую" + " пару" + "; "
            
    if z == 0: 
        print(f"В данный день - {k} - все запрошенные пользователи заняты")
        
    else: 
        final2 = final[:-2] + '.'
        print(f'В данный день - {k} - все запрошенные пользователи свободны в {final2}')
    

<ipython-input-6-515e333d2207>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  br.find_element_by_css_selector('#start').click() #кликает на дату
<ipython-input-6-515e333d2207>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  br.find_element_by_css_selector('#start').clear()#пока не удаляет дату, которую руз сам вставляет
<ipython-input-6-515e333d2207>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  br.find_element_by_css_selector('#start').send_keys(i) #вставляет нашу дату (сейчас криво вставится, все исправится!)
<ipython-input-6-515e333d2207>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  br.find_element_by_css_selector('#start').clear() #теперь наконец-то удается убрать все
<ipython-input-6-515e333d2207>:14: DeprecationWarning: find_element_by_* commands are deprecate

В данный день - 24.01.2022 - все запрошенные пользователи свободны в 6-ую пару; 7-ую пару.
В данный день - 25.01.2022 - все запрошенные пользователи свободны в 1-ую пару; 5-ую пару; 6-ую пару; 7-ую пару.


### Шаг 4. Результаты в виде таблицы

Запустите следующую ячейку, чтобы увидеть результаты работы программы в виде удобной таблицы по всем запрошенным дням.

In [7]:
list_date_df = []
for k, v in extra.items():
    date_df = []
    for i in range(len(v)): 
        if v[i] == "0":
            date_df.append("Свободно")
        else:
            date_df.append("-") 
    list_date_df.append(date_df)       

#работа с датафреймом, создаём его на основе списка дат, присоединяем
#список выше и приводим в удобный вид
df = pd.DataFrame(list_date_df)
df["Даты"] = dati
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df = df.rename(columns={0: '1 пара, 09:30 - 10:50', 
                        1:'2 пара, 11:10 - 12:30', 
                        2:'3 пара, 13:00 - 14:20', 
                        3:'4 пара, 14:40 - 16:00', 
                        4:'5 пара, 16:20 - 17:40', 
                        5:'6 пара, 18:10 - 19:30', 
                        6:'7 пара, 19:40 - 21:00'})
df = df.set_index('Даты').transpose()

df

Даты,24.01.2022,25.01.2022
"1 пара, 09:30 - 10:50",-,Свободно
"2 пара, 11:10 - 12:30",-,-
"3 пара, 13:00 - 14:20",-,-
"4 пара, 14:40 - 16:00",-,-
"5 пара, 16:20 - 17:40",-,Свободно
"6 пара, 18:10 - 19:30",Свободно,Свободно
"7 пара, 19:40 - 21:00",Свободно,Свободно


### Шаг 5 (опциональный). Выгрузка результатов в таблицу Excel

Если Вы хотите сохранить результаты работы программы в таблицу Excel (формат .xslx), запустите следующую ячейку.

* Укажите путь к папке, в которую хотите сохранить результаты, в формате 'C:\Users' (без кавычек), нажмите Enter
* Укажите название, под которым хотите сохранить файл, нажмите Enter

*В названии можно использовать русские и латинские буквы, цифры и символы, кроме „<”, „>”, „:”, „«”, „/”, „\”, „|”, „?” и „*”.*

*Длина названия не должна превышать 255 символов.*

In [ ]:
save_path = input("Пожалуйста, укажите путь к папке, в которую хотите сохранить результаты: ")
file_name = input("Пожалуйста, укажите название, под которым хотите сохранить результаты: ")
save_path_file = save_path + '\\' + file_name + '.xlsx'
df.to_excel(save_path_file)
print("\nВаш файл сохранён по указанному пути")